In [1]:
##############################################################
## ETH Zuerich, September 2018
## Author: Anna Stopka, anna.stopka"at"bsse.ethz.ch
## 
## software: Python 2.7
##
##############################################################
## Code: simulation of stem cell population dynamics, 
##       stem cells can divide symmetrically or do not divide, 
##       single cell dynamics: Markov process
##
##############################################################

In [2]:
##############################################################
## 1) import libraries and define functions
##############################################################

import numpy as np
from matplotlib.pyplot import scatter

def gen_alpheps(amean, asigma, emean, esigma, corr, n):
    """  
    input: amean, asigma, emean, esigma, correlation value corr, size n
    output: two normal distributions being correlated
    """
    means = [amean, emean] 
    stds = [asigma, esigma]

    # covariance matrix for two variables with correlation
    covs = [[stds[0]**2 ,           stds[0]*stds[1]*corr],
            [stds[0]*stds[1]*corr,           stds[1]**2]]
    
    m = np.random.multivariate_normal(means, covs, n).T
    
    # make sure that no negative epsilon and no cell with a cell cycle time >100h
    for i in range(len(m[0])):
        if m[1][i] < 0.01:
            bool = True
            while bool:
                new = np.random.multivariate_normal(means, covs, 1).T
                if new[1] > 0.01:
                    m[0][i] = new[0]
                    m[1][i] = new[1]
                    bool = False
    
    combined = list(zip(m[0],m[1]))  
    return combined

def init_stemcellmatrix(cells,runtime):
    """
    initializes stem cell matrix
    input:
    - cells (int): initial stem cell population size
    - runtime (int): simulated time
    output:
    - apop (list): dimension (stemcells x runtime)
    """
    apop = np.zeros(shape=(cells,runtime))
    apop[:,0]=1
    return apop
    
def init_diffcellmatrix(cells,runtime):    
    """
    initializes differentiated cell matrix
    input:
    - cells (int): initial stem cell population number
    - runtime (int): simulated time
    output:
    - bpop (list): dimension (stemcells x runtime)
    """
    bpop = np.zeros(shape=(cells,runtime)) 
    return bpop

def init_stemcellbirth(apop):
    """
    initializes stem cell birth dictionary
    input:
    - apop (list): initial stem cell population matrix
    output:
    - births (dict): key = stem cell ID, value: birth time point
    """
    births={}
    for a in range(len(apop)):
        births[a] = 1
    return births

def iterate_cells(apop,bpop,distr1,births,runtime,amean,asigma,emean,esigma,c):
    """
    iteration of stem cell population dynamics over time:
    loop for each single cell over time using the core algorithm (Marcov process)
    input: 
    - apop (list): matrix for stem cells (cells x runtime)
    - bpop (list): matrix for differentiated cells (cells x runtime)
    - distr1 (list): initial probability distribution
    - births (dict): key = stem cell ID, value: birth time point
    - runtime (int): simulation runtime
    - amean (float): mean value of alpha
    - emean (float): mean value of epsilon
    - asigma (float): std of alpha
    - esigma (float): std of epsilon    
    output:
    - diffrate (float): differentiation rate value 
    - rob (bool): indicates robustness
    - overgrow (bool): indicates overgrowth in case of inrobustness
    - shrink (bool): indicates shrinkage in case of inrobustness 
    """  
    
    rob = True        # will be changed to False in case of inrobustness
    overgrow = False  # will be changed to True in case of overgrowth
    shrink = False    # will be changed to True in case of shrinkage
    checkinrob = True # if True, then check for inrobustness
    
    apop_init=len(apop) # initial stem cell population size
    
    count = 0
    while count < len(apop):

        a = count # recent cell ID   
        count +=1 # increment counter for next cell

        # probability set of the recent cell
        alpha = distr1[a][0]
        epsilon = distr1[a][1]
        
        #generate random numbers
        cellcyc = np.random.uniform(0,1,int(runtime))
        differen = np.random.uniform(0,1,int(runtime))

        # time iteration for recent cell, start is birth date of that cell:
        for t in range(births[a],int(runtime),1):
            
            # the value of the previous time step is copied (in case a stem cell remains)
            # these values will be changed in case of a division
            apop[a][t] = apop[a][t-1]
            bpop[a][t] = bpop[a][t-1]

            # core algorithm (Markov process):
            cc = cellcyc[t]
            if cc <= epsilon: # stem cell enters cell cycle
                diff = differen[t]
                if diff <= alpha: # stem cell differentiates
                    apop[a][t:] = 0
                    bpop[a][t] += 2
                    bpop[a][t:] = bpop[a][t]
                    break
                else: # stem cell maintains stemness
                    # generate new cell in population and set its birth date
                    apop = np.vstack([apop, np.zeros(runtime)])
                    bpop = np.vstack([bpop, np.zeros(runtime)])
                    apop[-1][t] += 1
                    births[(len(apop)-1)]=t+1
                    # generation of a new set of probabilities for the new born stem cell
                    distr2=[]
                    distr2 = gen_alpheps(amean,asigma, emean, esigma, c,1)[0]
                    distr1.append(distr2)

        # check after time iteration of every cell for overgrowth (to avoid long runs):
        apop_now = apop.sum(axis=0)[-1] # recent number of stem cells
        if (apop_now > 2*apop_init):
            overgrow = True
            count = len(apop)  # no time iteration for further cells
            rob=False
            checkinrob = False # no further need to check for shrinkage later
            break              # stop the run here

    # final number of stem cells and pre-differentiated cells
    apop_final = apop.sum(axis=0)[-1]
    bpop_final = bpop.sum(axis=0)[-1]
    
    # calculate performance:
    diffrate = bpop_final/(runtime*apop_init)

    # check in the end for shrinkage (this case has a short run time):
    if (checkinrob == True) and (apop_final <= 0.0):
        shrink = True
        rob=False
    
    return [diffrate,rob,overgrow,shrink]

In [4]:
##############################################################
## 2) define parameters
##############################################################

p = {'amean'  : 0.5   # mean value of alpha
    ,'asigma' : 0.08  # std of alpha
    ,'emean'  : 0.06  # mean value of epsilon
    ,'esigma' : 0.02  # std of epsilon
    ,'s'      : 50    # initial stem cell population size
    ,'corr'   : 0.0  # correlation
    ,'time'   : 100   # runtime in hours
    ,'delta_t': 1.0   # one iteration step in hours
    ,'numb'   : 10    # number of simulation sets
    ,'runs'   : 30    # runs per set
    }

In [5]:
##############################################################
## 3) run and evaluate simulations
##############################################################

D=[]
R=[]
O=[]
S=[]
        
for l in range(p['numb']):
    print "### set number ", l+1, " is running"
    differentiationrate =[]
    overgrow=0
    shrink=0

    distr = list(gen_alpheps(p['amean'],p['asigma'],p['emean'],p['esigma'],p['corr'],p['s']))

    for i in range(p['runs']):

        # generate probability set for initial population
        distr1 = []
        distr1 = distr[:]

        # initialize matrix for stem cell population and differentiated cells for the entire runtime
        apop=init_stemcellmatrix(p['s'],p['time'])
        bpop=init_diffcellmatrix(p['s'],p['time'])

        # initialize dictionary that keeps track of the timepoint a stem cell is born
        births=init_stemcellbirth(apop)

        # iterate over time for each stem cell of a population:
        l=[]
        l=iterate_cells(apop,bpop,distr1,births,p['time'],p['amean'],p['asigma'],p['emean'],p['esigma'],p['corr'])    

        # evaluate results:
        if l[1]:
            differentiationrate.append(l[0])
        if l[2]:
            overgrow += 1
        if l[3]:
            shrink += 1

    # append results of this set
    R.append(((float(p['runs'])-(overgrow+shrink))/p['runs'])*100.)      
    if len(differentiationrate)>0:
        D.append(np.mean(differentiationrate)/p["delta_t"])
    if (overgrow >0) or (shrink>0):
        O.append((float(overgrow)/(overgrow+shrink))*100.)
        S.append((float(shrink)/(overgrow+shrink))*100.)

# print results
print "####################"
print "average robustness: " ,  round(np.mean(R),2)  , "+-",  round(np.std(R),2)  , "%"
if len(D)>0:
    print "average differentiation rate: ", round(np.mean(D),4), "+-" , round(np.std(D),4)

if len(O)>0:
    print "inrobust cases: "
    print round(np.mean(O),2), "+-", round(np.std(O),2), "% overgrowth, "
    print round(np.mean(S),2) , "+-", round(np.std(S),2) , "% depletion "

### set number  1  is running
### set number  2  is running
### set number  3  is running
### set number  4  is running
### set number  5  is running
### set number  6  is running
### set number  7  is running
### set number  8  is running
### set number  9  is running
### set number  10  is running
####################
average robustness:  99.0 +- 1.53 %
average differentiation rate:  0.0575 +- 0.0025
inrobust cases: 
100.0 +- 0.0 % overgrowth, 
0.0 +- 0.0 % depletion 
